In [ ]:
%load_ext watermark
%watermark -v -p numpy,pandas,matplotlib,torch


In [ ]:
# Loading the dataset
import pandas as pd

df=pd.read_csv('./assets/xor.csv')
df

In [ ]:
X = df[['x1','x2']].values
y = df['class label'].values

In [ ]:
from sklearn.model_selection import train_test_split

# split train test
X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.15, random_state=1, shuffle=True, stratify=y)

In [ ]:
# split on train to validate performance

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

In [ ]:
print(f"Training set size: {len(X_train)}")
print(f"Val set size: {len(X_val)}")
print(f"Test size : {len(X_test)}")

In [ ]:
import numpy as np

print(f"Training labels: {np.bincount(y_train)}")
print(f"Val labels: {np.bincount(y_val)}")
print(f"Test labels : {np.bincount(y_test)}")

In [ ]:
# visualize the dataset
import matplotlib.pyplot as plt
plt.plot(
    X_train[y_train == 0, 0],
    X_train[y_train == 0, 1],
    marker="D",
    markersize=10,
    linestyle="",
    label="Class 0",
)

plt.plot(
    X_train[y_train == 1, 0],
    X_train[y_train == 1, 1],
    marker="^",
    markersize=13,
    linestyle="",
    label="Class 1",
)

# plt.plot([x1_min, x1_max], [x2_min, x2_max], color="k")

plt.legend(loc=2)

plt.xlim([-5, 5])
plt.ylim([-5, 5])

plt.xlabel("Feature $x_1$", fontsize=12)
plt.ylabel("Feature $x_2$", fontsize=12)

plt.grid()
plt.show()

In [ ]:
import torch

class PytorchMPL(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super().__init__()

        self.all_layers = torch.nn.Sequential(

            #first hidden layer
            torch.nn.Linear(num_features, 25),
            torch.nn.ReLU(),

            #Second hidden layer
            torch.nn.Linear(25, 15),
            torch.nn.ReLU(),

            #output layer
            torch.nn.Linear(15, num_classes),
        )

    def forward(self, x):
        logits = self.all_layers(x)
        return logits




In [ ]:
# define the dataloader

from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.int64)

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):

        x = self.features[idx]
        y = self.labels[idx]

        return x, y

train_ds = MyDataset(X_train, y_train)
val_ds = MyDataset(X_val, y_val)
test_ds = MyDataset(X_test, y_test)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)




In [ ]:
# Training loop

def compute_accuracy(model, dataloader):

    correct = 0.0
    total_examples = 0.0

    for index, (features, labels) in enumerate(dataloader):

        # with torch.no_grad():
        with torch.inference_mode():  #same as no grad
            logits = model(features)

        predictions = torch.argmax(logits, dim=1) # because it one hot encodes y get its index for class label
        compare = predictions == labels
        correct += torch.sum(compare)
        total_examples += len(compare)

    return correct / total_examples

In [ ]:
import torch.nn.functional as F

torch.manual_seed(1)
model = PytorchMPL(num_features=2, num_classes=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

n_epochs = 10

for epoch in range(n_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_dl):

        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        # logging
        print(
            f"Epoch {epoch + 1:03d} / {n_epochs:03d} - "
            f"Batch {batch_idx:03d} / {len(train_dl):03d} -"
            f"Loss: {loss.item():.4f}"
              )

    train_acc = compute_accuracy(model, train_dl)
    val_acc = compute_accuracy(model, val_dl)
    print(f"Train accuracy: {train_acc:.4f} - Val accuracy: {val_acc:.4f}")






# Evaluate our model

In [ ]:
train_acc = compute_accuracy(model, train_dl)
val_acc = compute_accuracy(model, val_dl)
test_acc = compute_accuracy(model, test_dl)

print(f"Train accuracy: {train_acc:.4f} - Val accuracy: {val_acc:.4f} - Test accuracy: {test_acc:.4f}")

In [ ]:
# Visualize the decision boundary

from matplotlib.colors import ListedColormap


def plot_decision_regions(X, y, classifier, resolution=0.02):
    # setup marker generator and color map
    markers = ('D', '^', 'x','s','v')
    colors = ('C0', 'C1', 'C2','C3','C4')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    tensor = torch.tensor(np.array([xx1.ravel(), xx2.ravel()]).T).float()
    logits = classifier.forward(tensor)
    # Use torch.argmax instead of np.argmax to avoid numpy conversion
    Z = torch.argmax(logits.detach(), axis=1)
    Z = Z.reshape(xx1.shape)

    # Convert to numpy only for matplotlib plotting
    print(Z.detach().shape)
    Z_numpy = Z.numpy()

    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, color=colors[idx],
                    marker=markers[idx],
                    label=cl)


plot_decision_regions(X_train, y_train, model)


In [ ]:
torch.tensor([2]).numpy()